# <font color='black'> Методы анализа неоднородных данных и паттерн-анализ. Практическое занятие 3 </font>
## <font color='black'> Анализ панельных данных. Модели с фиксированными эффектами (FE-models) и модели со случайными эффектами (Часть 1) </font>
Типичные панельные данные имеют как пространственное, так и временное измерение. К примеру, наблюдения для одной страны представлены за несколько лет. При такой структуре данных наивно было бы полагать, что условие о независимости наблюдений соблюдается. Наблюдения "внутри" одной пространственной единицы, разумеется, связаны. В этом случае мы не можем применять классическую линейную регрессию. В качестве альтернативы выступают широко применяемые модели с фиксированными эффектами.

Кратко о данных, с которыми мы сегодня будем работать.

*Источник: Массив данных основан на идее исследования Ehrlich I., Lui F. (1999) Bureaucratic Corruption and Endogenous Economic Growth. The Journal of Political Economy, 107 (6), pp. 270 – 293.*

* country -  Страна
* countrygroup - Страны в датасете разделены на три группы. Первая группа (countrygroup = 1) включает Австралию, Канаду, Японию, Новую Зеландию и США. Вторая группа (countrygroup = 2) включает страны Восточной Европы (Болгария, Хорватия, Чешская Республика, Эстония, Венгрия, Латвия, Польша, Словения, Словакия). В третью группу в основном входят страны Западной Европы
* year - Год. Данные охватывают временной промежуток с 2006 по 2015 год
* pol_stab - Политическая стабильность и отсутствие насилия/терроризма. Отражает склонность к политической нестабильности и/или политически мотивированному насилию, включая терроризм. Значения показателя варьируются от −2.5 до 2.5, причем более высокие значения означают более высокий уровень политической стабильности. Источником данных являются Индикаторы качества государственного управления (WGI). В рамках анализа будем использовать этот показатель в качестве зависимой переменной
* con_cor - Показатель контроля коррупции. Отражает восприятие степени использования государственной власти для личной выгоды, включая как мелкие, так и крупные формы коррупции. Показатель изменяется от −2.5 до 2.5, причем более высокие зна-чения соответствуют более низкому уровню коррупции (т.е. более высокому контролю). Источником данных является WGI. В рамках анализа будем использовать эту переменную в качестве ключевого предиктора
* herfgov_DPI - Индекс концентрации Херфиндаля. В контексте данного исследования он используется для измерения уровня конкуренции между политическими партиями. Шкала варьируется от 0 до 1, где 1 означает отсутствие конкуренции. В рамках анализа будем использовать как контрольную переменную
* govt_consump_WDI - Государственные расходы на конечное потребление (в процентах от ВВП). В рамках анализа будем использовать как контрольную переменную


Подгрузим необходимые библиотеки и откроем массив "MApolit_lab3.dta".

In [ ]:
import pandas as pd
import statsmodels.formula.api as statf
import numpy as np
import numpy.linalg as la
!pip install linearmodels
from linearmodels import PanelOLS
from linearmodels import RandomEffects
from scipy import stats

In [ ]:
lab3 = pd.read_stata("MApolit_lab3.dta")
lab3 = lab3.dropna()
lab3.head(20)

Выведем описательные статистики для ключевых переменных, но по подгруппам стран.

In [ ]:
lab3.groupby('countrygroup')['pol_stab'].describe()

In [ ]:
lab3.groupby('countrygroup')['con_cor'].describe()

В нашей первой модели мы учтем разные "стартовые" условия. Для этого добавим дамми-переменные на страну. Кроме того, пока включим только один предиктор - показатель контроля коррупции. Проинтерпретируйте оценки model1. Почему в данном случае мы получаем большой R-squared? Можно ли на него полагаться?

In [ ]:
model1 = statf.ols(formula = 'pol_stab ~ con_cor + C(country)', data = lab3).fit()
print(model1.summary())

Как Вы уже могли заметить выше, Python по умолчанию в качестве базовой категории (относительно которой происходит сравнение) выбирает первую пространственную единицу. При этом не всегда удобно интерпретировать относительно первой категории. Ниже рассмотрим, как можно изменить базовую категорию.

* Что в выдаче изменилось? Изменилась ли оценка коэффициента при контроле коррупции?
* Проинтерпретируйте оценки model1_2

In [ ]:
model1_2 = statf.ols(formula = 'pol_stab ~ con_cor + C(country, Treatment("Denmark"))', data = lab3).fit()
print(model1_2.summary())

На следующем шаге оценим модель с внутригрупповым преобразованием. Вспомните, в чем ее отличие от LSDV-модели (модели с дамми-переменными), почему часто используется именно такая спецификация с внутригрупповым преобразованием. Обратите внимание на то, что для того, чтобы в Python оценить FE-модели и RE-модели, нужно предварительно указать, какая переменная показывает пространственное измерение, а какая - временное.
* Проинтерпретируйте оценки model2
* Проинтерпретируйте значения F-test for Poolability

In [ ]:
lab3_index = lab3.set_index(['country', 'year'])
model2 = PanelOLS.from_formula('pol_stab ~ con_cor + EntityEffects', data = lab3_index).fit()
print(model2)

Получим теперь ту же самую оценку коэффициента при показателе контроля коррупции, однако посредством деления массива на подвыборки. Страны и будут составлять наши подвыборки. На первом шаге посчитаем разброс значений контроля коррупции по подгруппам.

In [ ]:
n = lab3.groupby('country', sort = False).size()
varbygroups = lab3.groupby('country', sort = False).var().con_cor*(n-1)

varbygroups.sort_values(ascending = False)

На втором шаге оценим парную линейную регрессию политической стабильности на уровень коррупции на подвыборках разных стран и сохраним оценки коэффициентов при предикторе.

In [ ]:
coefconcor = []

for i in lab3.country.unique():
  subsample = lab3[lab3.country == i]
  coefconcor.append(statf.ols(formula = 'pol_stab ~ con_cor', data = subsample).fit().params[1])

Третий шаг - посчитаем сумму взвешенных оценок коэффициентов. Вес для i-ой страны - отношение внутригруппового разброса значений показателей контроля коррупции i-ой страны к сумму соответствующих внутригрупповых разбросов по всем странам.

И вуаля! Получили ту же оценку коэффициента посредством процедуры взвешивания. Чем нам с практической точки зрения полезны эти результаты?

In [ ]:
weightedsum = sum(coefconcor*varbygroups/sum(varbygroups))
weightedsum

Давайте теперь расширим модель, добавив контрольные переменные.

In [ ]:
model3 = statf.ols(formula = 'pol_stab ~ con_cor + govt_consump_WDI + herfgov_DPI + C(country)', data = lab3).fit()
print(model3.summary())

Давайте адаптируем алгоритм взвешивания к случаю множественной регрессионной модели. В целом алгоритм остается тем же, но теперь нам предварительно нужно очистить вариацию зависимой переменной и интересующего нас предиктора от контрольных переменных. Для того, чтобы это сделать, нам сначала нужно оценить регрессию зависимой переменной на все контроли и сохранить остатки (то есть, информация, несвязанная с контролями). И таким же образом очищаем вариацию предиктора.

Как и в модели 2, сфокусируемся на оценке коэффициента при предикторе con_cor.

In [ ]:
lab3['y_resid'] = statf.ols(formula = 'pol_stab ~ govt_consump_WDI + herfgov_DPI + C(country)', data = lab3).fit().resid
lab3['x_resid'] = statf.ols(formula = 'con_cor ~ govt_consump_WDI + herfgov_DPI + C(country)', data = lab3).fit().resid

lab3.head()

Теперь упорядочим значения вариации для x_resid, для того, чтобы понять, какие страны оказались наиболее "весомыми" для получения оценки коэффициента при con_cor, а какие - наоборот, наименее значимыми.

In [ ]:
cleanvarbygroups = lab3.groupby('country', sort = False).var().x_resid*(n-1)

cleanvarbygroups.sort_values(ascending = False)

Посчитаем теперь взвешенную сумму оценок коэффициентов, полученных в результате оценивания моделей y_resid на x_resid по подгруппам. Убедимся, что в итоге это та же оценка коэффициента при con_cor, что мы получали и ранее в FE-модели.

In [ ]:
coefconcor_multiplereg = []

for i in lab3.country.unique():
  subsample = lab3[lab3.country == i]
  coefconcor_multiplereg.append(statf.ols(formula = 'y_resid ~ x_resid', data = subsample).fit().params[1])

In [ ]:
weightedsum1 = sum(coefconcor_multiplereg*cleanvarbygroups/sum(cleanvarbygroups))
weightedsum1

Теперь оценим ту же модель - model 3, но с внутригрупповым преобразованием. EntityEffects означает, что мы используем внутригрупповое преобразование по пространственным единицам, то есть, центрируем как зависимую переменную, так и предикторы по странам.

In [ ]:
model3_1 = PanelOLS.from_formula('pol_stab ~ con_cor + govt_consump_WDI + herfgov_DPI + EntityEffects', data = lab3_index).fit()
print(model3_1)